In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
sub_data = pd.read_csv('sampleSubmission.csv')
train_data

,id,date,speed
0,0,1/1/2017 0:00,43.002930
1,1,1/1/2017 1:00,46.118696
2,2,1/1/2017 2:00,44.294158
3,3,1/1/2017 3:00,41.067468
4,4,1/1/2017 4:00,46.448653
...,...,...,...
14001,14001,31/12/2018 12:00,19.865269
14002,14002,31/12/2018 15:00,17.820375
14003,14003,31/12/2018 16:00,12.501851
14004,14004,31/12/2018 18:00,15.979319


In [3]:
from wwo_hist import retrieve_hist_data
import urllib.request


frequency = 24
start_date = '01-Jan-2017'
end_date = '31-DEC-2018'
api_key = '8b9dd8eb5f174b05b0a143348200412'
location_list = ['hongkong']

hist_weather_data = retrieve_hist_data(api_key, location_list, start_date, end_date, frequency = 24, export_csv = True, store_df = True)



Retrieving weather data for hongkong


Currently retrieving data for hongkong: from 2017-01-01 to 2017-01-31
Time elapsed (hh:mm:ss.ms) 0:00:05.143720
Currently retrieving data for hongkong: from 2017-02-01 to 2017-02-28
Time elapsed (hh:mm:ss.ms) 0:00:09.428544
Currently retrieving data for hongkong: from 2017-03-01 to 2017-03-31
Time elapsed (hh:mm:ss.ms) 0:00:12.790079
Currently retrieving data for hongkong: from 2017-04-01 to 2017-04-30
Time elapsed (hh:mm:ss.ms) 0:00:17.804990
Currently retrieving data for hongkong: from 2017-05-01 to 2017-05-31
Time elapsed (hh:mm:ss.ms) 0:00:23.199579
Currently retrieving data for hongkong: from 2017-06-01 to 2017-06-30
Time elapsed (hh:mm:ss.ms) 0:00:27.215686
Currently retrieving data for hongkong: from 2017-07-01 to 2017-07-31
Time elapsed (hh:mm:ss.ms) 0:00:30.947730
Currently retrieving data for hongkong: from 2017-08-01 to 2017-08-31
Time elapsed (hh:mm:ss.ms) 0:00:36.103060
Currently retrieving data for hongkong: from 2017-09-01 to 2017

timeout: timed out

In [ ]:
dateparse = lambda dates: datetime.strptime(dates, '%Y-%m-%d')

df_weather = pd.read_csv('hongkong.csv', parse_dates=['date_time'],
                         index_col='date_time', date_parser=dateparse)
weather_features = df_weather[['tempC','winddirDegree',
                               'windspeedKmph','cloudcover', 
                               'WindChillC'，'humidity','visibility']]

In [ ]:
def get_holiday(s):
  s=str(s)
  list = re.split(' ', s)
  days = str(datetime.strptime(list[0],"%d/%m/%Y"))
  day = re.split(' ', days)[0]
  return day

train_data["hol"] = train_data["date"].apply(lambda x: get_holiday(x))
train_data["hol"]


def get_hour(s):
  s=str(s)
  list = re.split(' |:',s)
  return list[1]

def get_month(s):
  s=str(s)
  list = re.split('/',s)
  return list[1]

def get_year(s):
  s=str(s)
  list = re.split('/| ',s)
  return list[2]

def get_day(s):
  s=str(s)
  list = re.split('/| ',s)
  return list[0]

def get_week(s):
  s=str(s)
  list = re.split(' ', s)
  week = datetime.strptime(list[0],"%d/%m/%Y").weekday()
  return str(week)

In [ ]:
holidays = ['2017-01-02','2017-01-28','2017-01-30','2017-01-31',
            '2017-04-04','2017-04-14','2017-04-15','2017-04-17',
            '2017-05-01','2017-05-03','2017-05-30','2017-07-01',
            '2017-10-02','2017-10-05','2017-10-28','2017-12-25',
            '2017-12-26','2018-01-01','2018-02-16','2018-02-17',
            '2018-02-18','2018-03-30','2018-03-31','2018-04-02',
            '2018-04-05','2018-05-01','2018-05-22','2018-06-18','2018-07-02',
            '2018-09-25','2018-10-01','2018-10-17','2018-12-25','2018-12-26']

def if_holiday(s):
  s=str(s)
  list = re.split(' ', s)
  days = str(datetime.strptime(list[0],"%d/%m/%Y"))
  day = re.split(' ', days)[0]
  if day in holidays:
    return 1
  return 0

In [ ]:
train_data["holiday"] = train_data["date"].apply(lambda x: get_holiday(x))
train_data["hour"] = train_data["date"].apply(lambda x : get_hour(x))
train_data["year"] = train_data["date"].apply(lambda x : get_year(x))
train_data["weekday"] = train_data["date"].apply(lambda x: get_week(x))
train_data["month"] = train_data["date"].apply(lambda x : get_month(x))
train_data["day"] = train_data["date"].apply(lambda x : get_day(x))

In [ ]:
C = []
vis = []
wind_dir = []
wind_speed = []
hum = []
cloud = []
Wind = []

for i in train_data['date']:
# transform the type from date to datetime
  string_date = str(i)
  string_date=re.split(" ",string_date)[0]
  date = datetime.strptime(string_date, '%d/%m/%Y')
  C.append(weather_features['C'][date])
  vis.append(weather_features['vis'][date])
  wind_dir.append(weather_features['wind_dir'][date])
  wind_speed.append(weather_features['wind_speed'][date])
  hum.append(weather_features['hum'][date])
  cloud.append(weather_features['cloud'][date])
  Wind.append(weather_features['Wind'][date])

train_data['vis'] = vis
train_data['wind_dir'] = wind_dir
train_data['Wind'] = Wind
train_data['hum'] = hum
train_data['cloud'] = cloud   
train_data['wind_speed'] = wind_speed
train_data['C'] = C

train_data

In [ ]:
train_clean = train_data.drop(["date"], axis=1)
train_clean

In [ ]:
train_clean.to_csv("train_Aftercleane_data.csv", index=False)

In [ ]:
test_data["hour"] = test_data["date"].apply(lambda x : get_hour(x))
test_data["weekday"] = test_data["date"].apply(lambda x: get_week(x))
test_data["holiday"] = test_data["date"].apply(lambda x: get_holiday(x))
test_data["month"] = test_data["date"].apply(lambda x : get_month(x))
test_data["day"] = test_data["date"].apply(lambda x : get_day(x))
test_data["year"] = test_data["date"].apply(lambda x : get_year(x))

test_data

In [ ]:
C = []
vis = []
wind_dir = []
wind_speed = []
hum = []
cloud = []
Wind = []

for i in tset_data['date']:
# transform the type from date to datetime
  string_date = str(i)
  string_date=re.split(" ",string_date)[0]
  date = datetime.strptime(string_date, '%d/%m/%Y')
  C.append(weather_features['C'][date])
  vis.append(weather_features['vis'][date])
  wind_dir.append(weather_features['wind_dir'][date])
  wind_speed.append(weather_features['wind_speed'][date])
  hum.append(weather_features['hum'][date])
  cloud.append(weather_features['cloud'][date])
  Wind.append(weather_features['Wind'][date])

test_data['vis'] = vis
test_data['wind_dir'] = wind_dir
test_data['Wind'] = Wind
test_data['hum'] = hum
test_data['cloud'] = cloud   
test_data['wind_speed'] = wind_speed
test_data['C'] = C

train_data

In [ ]:
test_data["speed"] = 0
test_clean = test_data.drop(["date"], axis=1)

test_clean

In [ ]:
test_clean.to_csv("test_Afterclean_data.csv", index=False)